In [8]:
import XLSX
xf = XLSX.readxlsx("datasett.xlsx")

XLSXFile("datasett.xlsx") containing 5 Worksheets
            sheetname size          range        
-------------------------------------------------
                    t 50x4          A1:D50       
                    k 12x6          A1:F12       
                    i 28x4          A1:D28       
                    n 9x5           A1:E9        
                    o 540x27        A1:AA540     


In [9]:
using DataFrames, XLSX
df1 = DataFrame(XLSX.readtable("datasett.xlsx", "t"))
df2 = DataFrame(XLSX.readtable("datasett.xlsx", "k"))
df3 = DataFrame(XLSX.readtable("datasett.xlsx", "i"))
df4 = DataFrame(XLSX.readtable("datasett.xlsx", "n"))
df5 = DataFrame(XLSX.readtable("datasett.xlsx", "o"))

Row,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
using JuMP, GLPK ,CPLEX
m = Model(CPLEX.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: CPLEX

In [11]:
t = select(df1, :t) # time index
T = nrow(df1)
P = select(df1, :P)
S = select(df1, :S)

Row,S
,Any
1,0.0243995
2,0.0243995
3,0.0243995
4,0.0243995
5,0.0235639
6,0.0235639
7,0.0235639
8,0.0235639
9,0.0235639


In [13]:
P[:,1] = convert(Vector{Float64}, df1[1:49,:P]) # time t의 구매가격 
S[:,1] = convert(Vector{Float64}, df1[1:49,:S]) # time t의 판매가격

49-element Vector{Float64}:
 0.02439952
 0.02439952
 0.02439952
 0.02439952
 0.02356392
 0.02356392
 0.02356392
 0.02356392
 0.02356392
 0.02356392
 0.02356392
 0.02356392
 0.02439952
 ⋮
 0.09475704
 0.09475704
 0.09475704
 0.09475704
 0.06801784
 0.06801784
 0.06801784
 0.02439952
 0.02439952
 0.02439952
 0.02439952
 0.02439952

In [14]:
n = select(df4, :n) # charger index
N = nrow(df4)
α = select(df4, :α) # charging power
β = select(df4, :β) # discharging power
ρ_ch = select(df4, :ρ_ch) # charger n의 충전 효율
ρ_dis = select(df4, :ρ_dis) # charger n의 방전 효율

Row,ρ_dis
,Any
1,0.85
2,0.85
3,0.85
4,0.85
5,0.85
6,0.85
7,0.85
8,0.85


In [15]:
α[:,1] = convert(Vector{Float64}, df4[1:8,:α]) # charger n의 충전power
β[:,1] = convert(Vector{Float64}, df4[1:8,:β]) # charger n의 방전power
ρ_ch[:,1] = convert(Vector{Float64}, df4[1:8,:ρ_ch]) # charger n의 충전 효율
ρ_dis[:,1] = convert(Vector{Float64}, df4[1:8,:ρ_dis]) # charger n의 방전 효율

8-element Vector{Float64}:
 0.85
 0.85
 0.85
 0.85
 0.85
 0.85
 0.85
 0.85

In [16]:
k = select(df2, :k) # bus index
K = nrow(df2)
C_bat = 88 # bus k의 배터리 capacity
E_0 = 0.5 # bus k의 초기 에너지 수준
E_min = 0.2 # bus k의 최소 에너지 수준
E_max = 0.8 # bus k의 최대 에너지 수준
E_end = 0.2 # bus k의 하루 운행 종료 후 최소 에너지 수준
U = 135 # transformer capa

135

In [17]:
i = select(df3, :i) # trip index
I = nrow(df3)
T_start = select(df3, :T_start) # time t에 출발
T_end = select(df3, :T_end) # time t에 도착
γ = select(df3, :γ) # 시간당 trip i의 평균 에너지 소비 수준

Row,γ
,Any
1,9.084
2,9.084
3,9.084
4,9.084
5,9.084
6,9.084
7,9.084
8,9.084
9,9.084


In [18]:
T_start[:,1] = convert(Vector{Int}, df3[1:27,:T_start]) # trip i의 시작
T_end[:,1] = convert(Vector{Int}, df3[1:27,:T_end]) # trip i의 종료
γ[:,1] = convert(Vector{Float64}, df3[1:27,:γ]) # trip i의 consumption

27-element Vector{Float64}:
 9.084
 9.084
 9.084
 9.084
 9.084
 9.084
 9.084
 9.084
 9.084
 9.084
 9.084
 9.084
 9.084
 ⋮
 9.084
 9.084
 9.084
 9.084
 9.084
 9.084
 9.084
 9.084
 9.084
 9.084
 9.084
 9.084

In [37]:
@variable(m, b[1:K,1:I,1:T], Bin) # bus k가 trip i를 serving 하는지 여부 
@variable(m, x[1:K,1:N,1:T], Bin) # bus k가 charger n에서 time t에 charging 여부
@variable(m, y[1:K,1:N,1:T], Bin) # bus k가 charger n에서 time t에 discharging 여부
@variable(m, c[1:K,1:T], Bin) # bus k의 time t에 charging, discharging 여부

LoadError: An object of name b is already attached to this model. If this
    is intended, consider using the anonymous construction syntax, e.g.,
    `x = @variable(model, [1:N], ...)` where the name of the object does
    not appear inside the macro.

    Alternatively, use `unregister(model, :b)` to first unregister
    the existing name from the model. Note that this will not delete the
    object; it will just remove the reference at `model[:b]`.


In [12]:
@variable(m, e[1:K,1:T]>=0) # time t에서 bus k의 에너지 수준 
@variable(m, w_buy[1:T]>=0) # time t에 충전량
@variable(m, w_sell[1:T]>=0) # time t에 방전량

49-element Vector{VariableRef}:
 w_sell[1]
 w_sell[2]
 w_sell[3]
 w_sell[4]
 w_sell[5]
 w_sell[6]
 w_sell[7]
 w_sell[8]
 w_sell[9]
 w_sell[10]
 w_sell[11]
 w_sell[12]
 w_sell[13]
 ⋮
 w_sell[38]
 w_sell[39]
 w_sell[40]
 w_sell[41]
 w_sell[42]
 w_sell[43]
 w_sell[44]
 w_sell[45]
 w_sell[46]
 w_sell[47]
 w_sell[48]
 w_sell[49]

In [13]:
@objective(m, Max, sum(S[t,1] * w_sell[t] for t in 1:T) - sum(P[t,1] * w_buy[t] for t in 1:T))

0.02439952 w_sell[1] + 0.02439952 w_sell[2] + 0.02439952 w_sell[3] + 0.02439952 w_sell[4] + 0.02356392 w_sell[5] + 0.02356392 w_sell[6] + 0.02356392 w_sell[7] + 0.02356392 w_sell[8] + 0.02356392 w_sell[9] + 0.02356392 w_sell[10] + 0.02356392 w_sell[11] + 0.02356392 w_sell[12] + 0.02439952 w_sell[13] + 0.02439952 w_sell[14] + 0.02439952 w_sell[15] + 0.02439952 w_sell[16] + 0.06801784 w_sell[17] + 0.06801784 w_sell[18] + 0.09475704 w_sell[19] + 0.09475704 w_sell[20] + 0.09475704 w_sell[21] + 0.06801784 w_sell[22] + 0.06801784 w_sell[23] + 0.06801784 w_sell[24] + 0.06801784 w_sell[25] + 0.06801784 w_sell[26] + 0.06801784 w_sell[27] + 0.06801784 w_sell[28] + 0.06801784 w_sell[29] + 0.06801784 w_sell[30] + 0.06801784 w_sell[31] + 0.06801784 w_sell[32] + 0.06801784 w_sell[33] + 0.06801784 w_sell[34] + 0.06801784 w_sell[35] + 0.06801784 w_sell[36] + 0.09475704 w_sell[37] + 0.09475704 w_sell[38] + 0.09475704 w_sell[39] + 0.09475704 w_sell[40] + 0.09475704 w_sell[41] + 0.06801784 w_sell[42] + 0

In [14]:
@constraint(m, c1[k in 1:K, t in 1:T], sum(b[k ,i, t] for i in 1:I) + sum(x[k ,n, t] for n in 1:N) + sum(y[k ,n, t] for n in 1:N) <= 1)

11×49 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 c1[1,1] : b[1,1,1] + b[1,2,1] + b[1,3,1] + b[1,4,1] + b[1,5,1] + b[1,6,1] + b[1,7,1] + b[1,8,1] + b[1,9,1] + b[1,10,1] + b[1,11,1] + b[1,12,1] + b[1,13,1] + b[1,14,1] + b[1,15,1] + b[1,16,1] + b[1,17,1] + b[1,18,1] + b[1,19,1] + b[1,20,1] + b[1,21,1] + b[1,22,1] + b[1,23,1] + b[1,24,1] + b[1,25,1] + b[1,26,1] + b[1,27,1] + x[1,1,1] + x[1,2,1] + x[1,3,1] + x[1,4,1] + x[1,5,1] + x[1,6,1] + x[1,7,1] + x[1,8,1] + y[1,1,1] + y[1,2,1] + y[1,3,1] + y[1,4,1] + y[1,5,1] + y[1,6,1] + y[1,7,1] + y[1,8,1] <= 1.0                                              …  c1[1,49] : b[1,1,49] + b[1,2,49] + b[1,3,49] + b[1,4,49] + b[1,5,49] + b[1,6,49] + b[1,7,49] + b[1,8,49] + b[1,9,49] + b[1,10,49] + b[1,11,49] + b[1,12,49] + b[1,13,49] + b[1,14,49] + b[1,15,49] + b[1,16,49] + b[1,17,49] + b[1,18,49] + b[1,19,49] + b[1,20,49] + b[1,21,49] + b[1

In [15]:
@constraint(m, c2[i in 1:I,k in 1:K, t in T_start[i,1]:T_end[i,1] - 1],b[k, i, t + 1] >= b[k, i ,t])

JuMP.Containers.SparseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}, 3, Tuple{Int64, Int64, Int64}} with 297 entries:
  [10, 7, 23 ]  =  c2[10,7,23] : -b[7,10,23] + b[7,10,24] >= 0.0
  [11, 6, 24 ]  =  c2[11,6,24] : -b[6,11,24] + b[6,11,25] >= 0.0
  [12, 3, 25 ]  =  c2[12,3,25] : -b[3,12,25] + b[3,12,26] >= 0.0
  [14, 9, 27 ]  =  c2[14,9,27] : -b[9,14,27] + b[9,14,28] >= 0.0
  [15, 8, 28 ]  =  c2[15,8,28] : -b[8,15,28] + b[8,15,29] >= 0.0
  [16, 11, 29]  =  c2[16,11,29] : -b[11,16,29] + b[11,16,30] >= 0.0
  [16, 5, 29 ]  =  c2[16,5,29] : -b[5,16,29] + b[5,16,30] >= 0.0
  [17, 11, 30]  =  c2[17,11,30] : -b[11,17,30] + b[11,17,31] >= 0.0
  [17, 5, 30 ]  =  c2[17,5,30] : -b[5,17,30] + b[5,17,31] >= 0.0
  [18, 4, 31 ]  =  c2[18,4,31] : -b[4,18,31] + b[4,18,32] >= 0.0
  [19, 9, 32 ]  =  c2[19,9,32] : -b[9,19,32] + b[9,19,33] >= 0.0
                ⋮
  [22, 3, 35 ]  =  c2[22

In [16]:
@constraint(m, c3[i in 1:I, t in T_start[i,1]:T_end[i,1]],sum(b[k, i, t] for k in 1:K ) == 1)

JuMP.Containers.SparseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}, 2, Tuple{Int64, Int64}} with 54 entries:
  [1, 15 ]  =  c3[1,15] : b[1,1,15] + b[2,1,15] + b[3,1,15] + b[4,1,15] + b[5,1,15] + b[6,1,15] + b[7,1,15] + b[8,1,15] + b[9,1,15] + b[10,1,15] + b[11,1,15] == 1.0
  [10, 24]  =  c3[10,24] : b[1,10,24] + b[2,10,24] + b[3,10,24] + b[4,10,24] + b[5,10,24] + b[6,10,24] + b[7,10,24] + b[8,10,24] + b[9,10,24] + b[10,10,24] + b[11,10,24] == 1.0
  [11, 24]  =  c3[11,24] : b[1,11,24] + b[2,11,24] + b[3,11,24] + b[4,11,24] + b[5,11,24] + b[6,11,24] + b[7,11,24] + b[8,11,24] + b[9,11,24] + b[10,11,24] + b[11,11,24] == 1.0
  [11, 25]  =  c3[11,25] : b[1,11,25] + b[2,11,25] + b[3,11,25] + b[4,11,25] + b[5,11,25] + b[6,11,25] + b[7,11,25] + b[8,11,25] + b[9,11,25] + b[10,11,25] + b[11,11,25] == 1.0
  [12, 25]  =  c3[12,25] : b[1,12,25] + b[2,12,25] + b[3,12,25] + b[4,12,25] + 

In [17]:
@constraint(m, c4[n in 1:N, t in 1:T], sum(x[k,n,t] for k in 1:K) + sum(y[k,n,t] for k in 1:K) <= 1)

8×49 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 c4[1,1] : x[1,1,1] + x[2,1,1] + x[3,1,1] + x[4,1,1] + x[5,1,1] + x[6,1,1] + x[7,1,1] + x[8,1,1] + x[9,1,1] + x[10,1,1] + x[11,1,1] + y[1,1,1] + y[2,1,1] + y[3,1,1] + y[4,1,1] + y[5,1,1] + y[6,1,1] + y[7,1,1] + y[8,1,1] + y[9,1,1] + y[10,1,1] + y[11,1,1] <= 1.0  …  c4[1,49] : x[1,1,49] + x[2,1,49] + x[3,1,49] + x[4,1,49] + x[5,1,49] + x[6,1,49] + x[7,1,49] + x[8,1,49] + x[9,1,49] + x[10,1,49] + x[11,1,49] + y[1,1,49] + y[2,1,49] + y[3,1,49] + y[4,1,49] + y[5,1,49] + y[6,1,49] + y[7,1,49] + y[8,1,49] + y[9,1,49] + y[10,1,49] + y[11,1,49] <= 1.0
 c4[2,1] : x[1,2,1] + x[2,2,1] + x[3,2,1] + x[4,2,1] + x[5,2,1] + x[6,2,1] + x[7,2,1] + x[8,2,1] + x[9,2,1] + x[10,2,1] + x[11,2,1] + y[1,2,1] + y[2,2,1] + y[3,2,1] + y[4,2,1] + y[5,2,1] + y[6,2,1] + y[7,2,1] + y[8,2,1] + y[9,2,1] + y[10,2,1] + y[11,2,1] <= 1.0     c4[2,49] : x[1,2,4

In [18]:
@constraint(m, c5[k in 1:K, t in 2:T], e[k,t] == e[k,t-1] + sum(ρ_ch[n,1]*α[n,1]*x[k,n,t] for n in 1:N)- sum(γ[i,1]*b[k,i,t] for i in 1:I) - sum(1/ρ_dis[n,1]*β[n,1]*y[k,n,t] for n in 1:N))

2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, Base.OneTo(11)
    Dimension 2, 2:49
And data, a 11×48 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 c5[1,2] : 9.084 b[1,1,2] + 9.084 b[1,2,2] + 9.084 b[1,3,2] + 9.084 b[1,4,2] + 9.084 b[1,5,2] + 9.084 b[1,6,2] + 9.084 b[1,7,2] + 9.084 b[1,8,2] + 9.084 b[1,9,2] + 9.084 b[1,10,2] + 9.084 b[1,11,2] + 9.084 b[1,12,2] + 9.084 b[1,13,2] + 9.084 b[1,14,2] + 9.084 b[1,15,2] + 9.084 b[1,16,2] + 9.084 b[1,17,2] + 9.084 b[1,18,2] + 9.084 b[1,19,2] + 9.084 b[1,20,2] + 9.084 b[1,21,2] + 9.084 b[1,22,2] + 9.084 b[1,23,2] + 9.084 b[1,24,2] + 9.084 b[1,25,2] + 9.084 b[1,26,2] + 9.084 b[1,27,2] - 18.7 x[1,1,2] - 18.7 x[1,2,2] - 18.7 x[1,3,2] - 18.7 x[1,4,2] - 18.7 x[1,5,2

In [19]:
@constraint(m, c6[t in 1:T], sum(sum(α[n,1] * x[k,n,t] for k in 1:K) for n in 1:N) == w_buy[t])

49-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 c6[1] : 22 x[1,1,1] + 22 x[2,1,1] + 22 x[3,1,1] + 22 x[4,1,1] + 22 x[5,1,1] + 22 x[6,1,1] + 22 x[7,1,1] + 22 x[8,1,1] + 22 x[9,1,1] + 22 x[10,1,1] + 22 x[11,1,1] + 22 x[1,2,1] + 22 x[2,2,1] + 22 x[3,2,1] + 22 x[4,2,1] + 22 x[5,2,1] + 22 x[6,2,1] + 22 x[7,2,1] + 22 x[8,2,1] + 22 x[9,2,1] + 22 x[10,2,1] + 22 x[11,2,1] + 22 x[1,3,1] + 22 x[2,3,1] + 22 x[3,3,1] + 22 x[4,3,1] + 22 x[5,3,1] + 22 x[6,3,1] + 22 x[7,3,1] + 22 x[8,3,1] + 22 x[9,3,1] + 22 x[10,3,1] + 22 x[11,3,1] + 22 x[1,4,1] + 22 x[2,4,1] + 22 x[3,4,1] + 22 x[4,4,1] + 22 x[5,4,1] + 22 x[6,4,1] + 22 x[7,4,1] + 22 x[8,4,1] + 22 x[9,4,1] + 22 x[10,4,1] + 22 x[11,4,1] + 22 x[1,5,1] + 22 x[2,5,1] + 22 x[3,5,1] + 22 x[4,5,1] + 22 x[5,5,1] + 22 x[6,5,1] + 22 x[7,5,1] + 22 x[8,5,1] + 22 x[9,5,1] + 22 x[10,5,1] + 22 x[11,5,1] + 7.4 x[1,6,1] + 7.4 x[2,6,1] + 7.4 x[3,6,

In [20]:
@constraint(m, c7[t in 1:T], sum(sum(β[n,1] * x[k,n,t] for k in 1:K) for n in 1:N) == w_sell[t])

49-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 c7[1] : 22 x[1,1,1] + 22 x[2,1,1] + 22 x[3,1,1] + 22 x[4,1,1] + 22 x[5,1,1] + 22 x[6,1,1] + 22 x[7,1,1] + 22 x[8,1,1] + 22 x[9,1,1] + 22 x[10,1,1] + 22 x[11,1,1] + 22 x[1,2,1] + 22 x[2,2,1] + 22 x[3,2,1] + 22 x[4,2,1] + 22 x[5,2,1] + 22 x[6,2,1] + 22 x[7,2,1] + 22 x[8,2,1] + 22 x[9,2,1] + 22 x[10,2,1] + 22 x[11,2,1] + 22 x[1,3,1] + 22 x[2,3,1] + 22 x[3,3,1] + 22 x[4,3,1] + 22 x[5,3,1] + 22 x[6,3,1] + 22 x[7,3,1] + 22 x[8,3,1] + 22 x[9,3,1] + 22 x[10,3,1] + 22 x[11,3,1] + 22 x[1,4,1] + 22 x[2,4,1] + 22 x[3,4,1] + 22 x[4,4,1] + 22 x[5,4,1] + 22 x[6,4,1] + 22 x[7,4,1] + 22 x[8,4,1] + 22 x[9,4,1] + 22 x[10,4,1] + 22 x[11,4,1] + 22 x[1,5,1] + 22 x[2,5,1] + 22 x[3,5,1] + 22 x[4,5,1] + 22 x[5,5,1] + 22 x[6,5,1] + 22 x[7,5,1] + 22 x[8,5,1] + 22 x[9,5,1] + 22 x[10,5,1] + 22 x[11,5,1] + 7.4 x[1,6,1] + 7.4 x[2,6,1] + 7.4 x[3,6,

In [21]:
@constraint(m, c8[k in 1:K], e[k,1] == E_0 * C_bat)

11-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 c8[1] : e[1,1] == 44.0
 c8[2] : e[2,1] == 44.0
 c8[3] : e[3,1] == 44.0
 c8[4] : e[4,1] == 44.0
 c8[5] : e[5,1] == 44.0
 c8[6] : e[6,1] == 44.0
 c8[7] : e[7,1] == 44.0
 c8[8] : e[8,1] == 44.0
 c8[9] : e[9,1] == 44.0
 c8[10] : e[10,1] == 44.0
 c8[11] : e[11,1] == 44.0

In [22]:
@constraint(m, c9[k in 1:K, t in 1:T], C_bat * E_min <= e[k,t] <= C_bat * E_max)

11×49 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.Interval{Float64}}, ScalarShape}}:
 c9[1,1] : e[1,1] in [17.6, 70.4]    …  c9[1,49] : e[1,49] in [17.6, 70.4]
 c9[2,1] : e[2,1] in [17.6, 70.4]       c9[2,49] : e[2,49] in [17.6, 70.4]
 c9[3,1] : e[3,1] in [17.6, 70.4]       c9[3,49] : e[3,49] in [17.6, 70.4]
 c9[4,1] : e[4,1] in [17.6, 70.4]       c9[4,49] : e[4,49] in [17.6, 70.4]
 c9[5,1] : e[5,1] in [17.6, 70.4]       c9[5,49] : e[5,49] in [17.6, 70.4]
 c9[6,1] : e[6,1] in [17.6, 70.4]    …  c9[6,49] : e[6,49] in [17.6, 70.4]
 c9[7,1] : e[7,1] in [17.6, 70.4]       c9[7,49] : e[7,49] in [17.6, 70.4]
 c9[8,1] : e[8,1] in [17.6, 70.4]       c9[8,49] : e[8,49] in [17.6, 70.4]
 c9[9,1] : e[9,1] in [17.6, 70.4]       c9[9,49] : e[9,49] in [17.6, 70.4]
 c9[10,1] : e[10,1] in [17.6, 70.4]     c9[10,49] : e[10,49] in [17.6, 70.4]
 c9[11,1] : e[11,1] in [17.6, 70.4]  …  c9[11,49] : e[11,49] in [17.6, 70.4]

In [23]:
@constraint(m, c10[t in 1:T], sum(sum(α[n,1] * x[k,n,t] for k in 1:K) for n in 1:N) + sum(sum(β[n,1] * y[k,n,t] for k in 1:K) for n in 1:N) <= U)

49-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 c10[1] : 22 x[1,1,1] + 22 x[2,1,1] + 22 x[3,1,1] + 22 x[4,1,1] + 22 x[5,1,1] + 22 x[6,1,1] + 22 x[7,1,1] + 22 x[8,1,1] + 22 x[9,1,1] + 22 x[10,1,1] + 22 x[11,1,1] + 22 x[1,2,1] + 22 x[2,2,1] + 22 x[3,2,1] + 22 x[4,2,1] + 22 x[5,2,1] + 22 x[6,2,1] + 22 x[7,2,1] + 22 x[8,2,1] + 22 x[9,2,1] + 22 x[10,2,1] + 22 x[11,2,1] + 22 x[1,3,1] + 22 x[2,3,1] + 22 x[3,3,1] + 22 x[4,3,1] + 22 x[5,3,1] + 22 x[6,3,1] + 22 x[7,3,1] + 22 x[8,3,1] + 22 x[9,3,1] + 22 x[10,3,1] + 22 x[11,3,1] + 22 x[1,4,1] + 22 x[2,4,1] + 22 x[3,4,1] + 22 x[4,4,1] + 22 x[5,4,1] + 22 x[6,4,1] + 22 x[7,4,1] + 22 x[8,4,1] + 22 x[9,4,1] + 22 x[10,4,1] + 22 x[11,4,1] + 22 x[1,5,1] + 22 x[2,5,1] + 22 x[3,5,1] + 22 x[4,5,1] + 22 x[5,5,1] + 22 x[6,5,1] + 22 x[7,5,1] + 22 x[8,5,1] + 22 x[9,5,1] + 22 x[10,5,1] + 22 x[11,5,1] + 7.4 x[1,6,1] + 7.4 x[2,6,1] + 7.4 x[3,

In [51]:
JuMP.optimize!(m)

println("b[1:K,1:I,1:T] = ", JuMP.value.(b))
println("x[1:K,1:N,1:T] = ", JuMP.value.(x))
println("y[1:K,1:N,1:T] = ", JuMP.value.(y)) 
println("c[1:K,1:T] = ", JuMP.value.(c))
println("e[1:K,1:T] = ", JuMP.value.(e))
println("w_buy[1:T] = ", JuMP.value.(w_buy))
println("w_sell[1:T] = ", JuMP.value.(w_sell))

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.05 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   = 7023.72 sec.
  Wait time (average)   =    0.09 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.05 ticks)

Consider using CPLEX node files to reduce memory usage.
CPLEX Error  1001: Out of memory.
b[1:K,1:I,1:T] = [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

x[1:K,1:N,1:T] = [1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0

 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 

y[1:K,1:N,1:T] = [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 

0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0

e[1:K,1:T] = [44.0 18.11764705882353 36.817647058823525 43.107647058823524 34.02364705882353 24.939647058823528 43.63964705882353 34.555647058823524 25.471647058823525 44.171647058823524 62.87164705882353 36.989294117647056 55.68929411764705 46.60529411764705 37.52129411764705 28.437294117647053 47.13729411764705 38.053294117647056 56.75329411764706 63.04329411764706 53.95929411764706 44.87529411764706 35.791294117647055 54.49129411764706 28.608941176470587 47.30894117647058 66.00894117647059 40.126588235294115 58.82658823529411 32.94423529411764 51.644235294117635 25.761882352941164 44.46188235294116 35.37788235294116 26.293882352941157 44.99388235294116 63.69388235294116 54.609882352941156 28.727529411764685 47.42752941176468 66.12752941176468 57.04352941176468 47.95952941176468 66.65952941176468 40.77717647058821 59.477176470588205 50.3931764705882 69.0931764705882 69.0931764705882; 44.0 18.11764705882353 36.817647058823525 55.51764705882353 46.433647058823524 65.13364705882353 56.0